In [1]:
from typing import TypedDict, Annotated
from langgraph.graph import add_messages, StateGraph, END
import os
from langchain_core.messages import HumanMessage
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langgraph.checkpoint.sqlite import SqliteSaver
import sqlite3


/home/mohitm/langgraph_revision/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

load_dotenv(override=True)


True

In [3]:

def get_groq_llm():
    return ChatOpenAI(
        model="openai/gpt-oss-20b",
        base_url="https://api.groq.com/openai/v1",
        api_key=os.getenv("GROQ_API_KEY"),
        temperature=0.7, max_tokens=2000
    )


In [4]:

sqlite_conn = sqlite3.connect("bot_checkpoint.sqlite", check_same_thread=False)

memory = SqliteSaver(sqlite_conn)

llm = get_groq_llm()


In [5]:

class BasicChatState(TypedDict): 
    messages: Annotated[list, add_messages]

def chatbot(state: BasicChatState): 
    return {
       "messages": [llm.invoke(state["messages"])]
    }


In [6]:

graph = StateGraph(BasicChatState)

graph.add_node("chatbot", chatbot)

graph.add_edge("chatbot", END)

graph.set_entry_point("chatbot")

app = graph.compile(checkpointer=memory)


In [7]:

config = {"configurable": {
    "thread_id": 1
}}

while True: 
    user_input = input("User: ")
    if(user_input in ["exit", "end"]):
        break
    else: 
        result = app.invoke({
            "messages": [HumanMessage(content=user_input)]
        }, config=config)

        print("AI: " + result["messages"][-1].content)

AI: 🎉 **Let’s play a fresh game of Tic‑Tac‑Toe!** 🎉  

I’ll keep the board for us.  
Here’s how we’ll play:

| What you type | What happens |
|---------------|--------------|
| **`X 5`** or **`O 7`** | Place your symbol (X or O) in the numbered square. |
| **`board`** | Show the current board. |
| **`help`** | Show this cheat‑sheet again. |
| **`quit`** | End the game. |

The squares are numbered like this:

```
 1 | 2 | 3
---+---+---
 4 | 5 | 6
---+---+---
 7 | 8 | 9
```

### 1️⃣ Choose Your Symbol

Do you want to be **X** or **O**?  
Type `X` or `O` and I’ll start the game (you’ll go first if you choose X, or I’ll go first if you choose O).

Let me know! 🚀
AI: Sure thing!  
Which symbol would you like to play as?  

- Type `X` if you want to be X (you’ll go first).  
- Type `O` if you want to be O (I’ll make the first move).

Let me know! 🚀
AI: **You placed an X in the center square (5).**  
I’ll play **O** and take the top‑left corner (1).

**Current board**

```
 O |   |  
---+---+